<a href="https://www.kaggle.com/code/oussamaelkhsoui/titanic?scriptVersionId=118512833" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import pandas as pd

In [ ]:
titanic_test = pd.read_csv("/kaggle/input/titanic-dataset/tested.csv")
titanic_train = pd.read_csv("/kaggle/input/titanic-dataset/Titanic-Dataset.csv")
titanic_train

In [ ]:
titanic_train.info()

In [ ]:
print(f'sex:\n{titanic_train["Sex"].value_counts()}')
print(f'Ticket:\n{titanic_train["Ticket"].value_counts()}')
print(f'Cabin        :\n{titanic_train["Cabin"].value_counts()}')
print(f'Embarked     :\n{titanic_train["Embarked"].value_counts()}')

In [ ]:
titanic_train.describe()

In [ ]:
titanic_test.describe()

In [ ]:
import matplotlib.pyplot as plt
titanic_train.hist(bins=50, figsize=(20,15))
# save_fig("attribute_histogram_plots")
plt.show()

In [ ]:
titanic_train_copy = titanic_train.copy()
titanic_train_copy["NSex"] = titanic_train_copy["Sex"]
titanic_train_copy.NSex[titanic_train_copy.NSex == "male"] = 1
titanic_train_copy.NSex[titanic_train_copy.NSex == "female"] = 0
titanic_train_copy.NSex = titanic_train_copy.NSex.astype(np.uint8)

titanic_test_copy = titanic_test.copy()
titanic_test_copy["NSex"] = titanic_test_copy["Sex"]
titanic_test_copy.NSex[titanic_test_copy.NSex == "male"] = 1
titanic_test_copy.NSex[titanic_test_copy.NSex == "female"] = 0
titanic_test_copy.NSex = titanic_test_copy.NSex.astype(np.uint8)

In [ ]:
for set_ in (titanic_train_copy, titanic_test_copy):
    set_.hist(bins=50, figsize=(20,15))
    # save_fig("attribute_histogram_plots")
    plt.show()

In [ ]:
for set_ in (titanic_train_copy, titanic_test_copy):
    set_.drop("PassengerId", axis=1, inplace=True)
    set_.drop("Sex", axis=1, inplace=True)
titanic_train_copy

In [ ]:
whole_data = pd.concat([titanic_train_copy, titanic_test_copy], ignore_index=True)
# whole_data
def att_propotions(attribute : str, data) -> pd.Series:
    return  data[attribute].value_counts()/ len(data)

In [ ]:
whole_data["Pclass"].value_counts()/ len(whole_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(whole_data, test_size=0.2, random_state=42)

comper_props_Pclass = pd.DataFrame({
    "overAll": att_propotions("Pclass", whole_data),
    "OurSplit": att_propotions("Pclass", titanic_test_copy),
    "Random": att_propotions("Pclass", test_set),
}).sort_index()
comper_props_Pclass

In [ ]:
comper_props_Sibsp = pd.DataFrame({
    "overAll": att_propotions("SibSp", whole_data),
    "OurSplit": att_propotions("SibSp", titanic_test_copy),
    "Random": att_propotions("SibSp", test_set),
}).sort_index()
comper_props_Sibsp

In [ ]:
comper_props_Parch = pd.DataFrame({
    "overAll": att_propotions("Parch", whole_data),
    "OurSplit": att_propotions("Parch", titanic_test_copy),
    "Random": att_propotions("Parch", test_set),
}).sort_index()
comper_props_Parch

In [ ]:
comper_props_NSex = pd.DataFrame({
    "overAll": att_propotions("NSex", whole_data),
    "OurSplit": att_propotions("NSex", titanic_test_copy),
    "Random": att_propotions("NSex", test_set),
}).sort_index()
comper_props_NSex

In [ ]:
comper_props_Survived = pd.DataFrame({
    "overAll": att_propotions("Survived", whole_data),
    "OurSplit": att_propotions("Survived", titanic_test_copy),
    "Random": att_propotions("Survived", test_set),
}).sort_index()
comper_props_Survived

# proposition:
The propotion of poeple who sevived is proportionate to the women and the propotion of poeple who does not sevived is proportionate to the men.
So les's suppose that NSex is a very important attribute to predict the survival. we may want to ensure that the test set is representative of the various of Sex in the whole dataset 

In [ ]:
whole_data["NSex"].hist(bins=50, figsize=(20, 15))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(whole_data, whole_data["NSex"]):
    strat_train_set = whole_data.loc[train_index]
    strat_test_set = whole_data.loc[test_index]

In [ ]:
strat_train_set

In [ ]:
comper_props_NSex = pd.DataFrame({
    "overAll": att_propotions("NSex", whole_data),
    "OurSplit": att_propotions("NSex", titanic_test_copy),
    "stratified": att_propotions("NSex", strat_test_set),
    "Random": att_propotions("NSex", test_set),
}).sort_index()
comper_props_NSex

In [ ]:
comper_props_Parch = pd.DataFrame({
    "overAll": att_propotions("Parch", whole_data),
    "OurSplit": att_propotions("Parch", titanic_test_copy),
    "stratified": att_propotions("NSex", strat_test_set),
    "Random": att_propotions("Parch", test_set),
}).sort_index()
comper_props_Parch

In [ ]:
comper_props_Survived = pd.DataFrame({
    "overAll": att_propotions("Survived", titanic_test_copy),
    "OurSplit": att_propotions("Survived", titanic_test_copy),
    "stratified": att_propotions("Survived", strat_test_set),
    "Random": att_propotions("Survived", test_set),
}).sort_index()
comper_props_Survived

In [ ]:
titanic = titanic_train_copy.copy()

In [ ]:
titanic.corr()

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(titanic, figsize=(12, 8))

# Prepare the Data for Machine Learning Algorithms

In [ ]:
titanic = titanic.drop("Survived", axis=1) # drop labels for training set
titanic_labels = titanic_train_copy["Survived"].copy()

In [ ]:
titanic.info()

In [ ]:
sample_incomplete_rows = titanic[titanic.isnull().any(axis=1)]
len(sample_incomplete_rows)

In [ ]:
from sklearn.impute import SimpleImputer

impoter = SimpleImputer(strategy="median")

In [ ]:
titanic_num = titanic.drop(["Embarked", "Cabin", "Name", "Ticket"], axis=1)
titanic_num

In [ ]:
impoter.fit(titanic_num)
impoter.statistics_

In [ ]:
X = impoter.transform(titanic_num)
X

In [ ]:
titanic_num_tr = pd.DataFrame(X, columns=titanic_num.columns, index=titanic.index)

sample_incomplete_rows_num_tr = titanic_num_tr[titanic_num_tr.isnull().any(axis=1)]
len(sample_incomplete_rows_num_tr)
print("")